In [168]:
import torch
import numpy as np
from scipy.integrate import quad
import sympy as smp
import matplotlib.pyplot as plt

In [169]:
def feature_map(x):
    # convert x to a pytorch tensor
    x_tensor = torch.tensor(x, dtype=torch.cfloat)

    # calculate the two components of the feature map, s1 and s2
    s1 = torch.exp(1j * (3*torch.pi/2) * x_tensor) * torch.cos(torch.pi/2 * x_tensor)
    s2 = torch.exp(-1j * (3*torch.pi/2) * x_tensor) * torch.sin(torch.pi/2 * x_tensor)

    feature_vector = torch.stack((s1, s2), dim=-1)

    return feature_vector


In [170]:
out = feature_map(0.4)

View vector as numpy array

In [171]:
out.numpy()

array([-0.25000003+0.76942086j, -0.18163565-0.55901694j], dtype=complex64)

Let's check that the output is normalised

In [172]:
out[0].abs().square().add(out[1].abs().square()).item()

0.9999999403953552

Okay now that we have the feature map, let's make our complex valued MPS. To keep things simple, we will consider the weight MPS for class A and class B separately. Our MPS has two sites and a local dimension of 2, so the first site has dimension $(1, d, \chi)$ and the second has dimension $(\chi, d, 1)$.

In [173]:
chi = 4
d = 2

In [174]:
torch.manual_seed(0)
t1 = torch.randn(1, d, chi, dtype=torch.cfloat, requires_grad=True)
t2 = torch.randn(chi, d, 1, dtype=torch.cfloat, requires_grad=True)

Now let's inspect the elements of the tensors:

In [175]:
t1.detach().numpy()

array([[[-0.796089  -0.8148417j , -0.17718582-0.30679867j,
          0.6001288 +0.4893244j , -0.22345477-1.495686j  ],
        [ 0.22788277-0.8933126j ,  0.24747548+0.21788357j,
          0.08474074+0.8751563j ,  0.7896807 -0.1748517j ]]],
      dtype=complex64)

In [176]:
t2.detach().numpy()

array([[[-0.9564706 -1.1992046j ],
        [ 0.40068242+0.5610952j ]],

       [[ 0.42344344-1.0996182j ],
        [-0.24137817+1.3102732j ]],

       [[ 0.530464  -0.414009j  ],
        [-0.1226102 +0.12973848j]],

       [[ 0.9824302 +1.1217078j ],
        [ 0.66913396-0.5965696j ]]], dtype=complex64)

Okay, now let's take a random product state and try and contract it with our 2 site MPS. We start by contracting the product state with the first MPS tensor t1 and then contract the state vector corresponding to the second site with t2. Then we contract the resulting tensors from these two operations over the shared bond dimension $\chi$.

In [177]:
p1 = feature_map(0.4) # construct our 2 product states
p2 = feature_map(0.1)

In [178]:
operation_1 = torch.einsum('ijk,j -> ik', t1, p1) # contract t1 with product state 1
operation_2 = torch.einsum('ijk,j -> ik', t2, p2)# contract t2 with product state 2
result = torch.einsum('ij, jk -> ik', operation_1, operation_2) # contract over the shared bond dimension chi

The result should be rank 1 (scalar)

In [179]:
result.item()

(0.24425482749938965+0.5001320242881775j)

Now let's convert this to our final output:

In [180]:
result.abs().item()

0.5565900206565857

This is our overlap $|f(x)|$. Now let's pretend that these product states correspond to a class 1 sample. Let's get the loss and gradient, then backprop using autograd.

In [181]:
loss_func = torch.nn.MSELoss()

Pretend that the target is 1 ie. overlap is 1

In [182]:
target = torch.tensor(1.0, dtype=torch.float)

In [183]:
result = result.abs()

In [184]:
result.squeeze() # squeeze to remove extra dimensions

tensor(0.5566, grad_fn=<SqueezeBackward0>)

Evaluate the loss function

In [185]:
loss = loss_func(result.squeeze(), target)

Get the gradient?

In [186]:
loss.backward()

Inspect the gradients for both tensors

In [187]:
t1.grad

tensor([[[-0.6079-0.8438j, -0.7655+0.1631j, -0.3822+0.2700j,  0.4340+0.8747j],
         [-0.0031+0.7556j,  0.5196+0.2310j,  0.3400+0.0045j,  0.1185-0.6995j]]])

In [188]:
t2.grad

tensor([[[-0.0026-0.4119j],
         [ 0.0525-0.0387j]],

        [[-0.1248-0.3544j],
         [ 0.0338-0.0490j]],

        [[-0.0332+0.1209j],
         [-0.0186+0.0070j]],

        [[-0.5663-0.6539j],
         [ 0.0311-0.1334j]]])

Manually update the two tensors. We'll use a learning rate of 0.01.

In [189]:
lr = 0.05

In [190]:
t1.grad

tensor([[[-0.6079-0.8438j, -0.7655+0.1631j, -0.3822+0.2700j,  0.4340+0.8747j],
         [-0.0031+0.7556j,  0.5196+0.2310j,  0.3400+0.0045j,  0.1185-0.6995j]]])

In [191]:
t1

tensor([[[-0.7961-0.8148j, -0.1772-0.3068j,  0.6001+0.4893j, -0.2235-1.4957j],
         [ 0.2279-0.8933j,  0.2475+0.2179j,  0.0847+0.8752j,  0.7897-0.1749j]]],
       requires_grad=True)

In [192]:
with torch.no_grad():
    t1.copy_(t1 - lr * t1.grad)
    t1.grad = None
    t2.copy_(t2 - lr * t2.grad)
    t2.grad = None

In [193]:
t1

tensor([[[-0.7657-0.7727j, -0.1389-0.3150j,  0.6192+0.4758j, -0.2452-1.5394j],
         [ 0.2280-0.9311j,  0.2215+0.2063j,  0.0677+0.8749j,  0.7838-0.1399j]]],
       requires_grad=True)

Now that we have updated both tensors, let's re-compute the loss

In [194]:
operation_1 = torch.einsum('ijk,j -> ik', t1, p1) # contract t1 with product state 1
operation_2 = torch.einsum('ijk,j -> ik', t2, p2) # contract t2 with product state 2
result = torch.einsum('ij, jk -> ik', operation_1, operation_2) # contract over the shared bond dimension chi

In [195]:
result.abs().squeeze()

tensor(0.8667, grad_fn=<SqueezeBackward0>)

Now let's scale up to a larger example.